In [1]:

# Delinquency Report (RC Package Automation)
# Developed by CD
# v1.8.1

import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Font, Border, Side, Alignment
# from openpyxl.styles.numbers import NumberFormat
import os
from datetime import datetime, timedelta, date
import json
from sqlalchemy import create_engine, text
import time
from pydantic import BaseModel, Field
from typing import List, Optional
from win32com.client import Dispatch



def retrieve_data():
    """
    Retrieve data from COCC database
    """
    class DatabaseHandler:
        """
        This class abstracts the connection to the database and allows a clean
        interface for the developer to use.

        """
        def __init__(self, tns_admin_path, credentials_path_db1, credentials_path_db2):
            """
            Args:
                tns_admin_path (str): Oracle driver path
                credentials_path_db1 (str): Database 1 credentials path
                credentials_path_db1 (str): Databsae 2 credentials path
            """
            os.environ['TNS_ADMIN'] = tns_admin_path

            with open(credentials_path_db1) as config_file:
                config1 = json.load(config_file)

            self.username1 = config1['username']
            self.password1 = config1['password']
            self.dsn1 = config1['dsn']

            with open(credentials_path_db2) as config_file:
                config2 = json.load(config_file)

            self.username2 = config2['username']
            self.password2 = config2['password']
            self.dsn2 = config2['dsn']

            self.connection_string1 = f'oracle+oracledb://{self.username1}:{self.password1}@{self.dsn1}'
            self.connection_string2 = f'oracle+oracledb://{self.username2}:{self.password2}@{self.dsn2}'

            self.engine1 = create_engine(self.connection_string1, max_identifier_length=128)
            self.engine2 = create_engine(self.connection_string2, max_identifier_length=128)

        def query(self, sql_query, engine=1):
            """
            This allows abstraction of the connection and the class
            so the developer can query a single table as a dataframe

            Args:
                sql_query (str): The query to SQL database is passed as a string
                engine (int): This selects the database. There are two engines:
                    1 -> R1625
                    2 -> COCC DataMart

            Returns:
                df: The SQL query is returned as a pandas DataFrame

            Usage:
                df = db_handler.query("SELECT * FROM DB.TABLE", engine=1)

                In this example, db_handler = DatabaseHandler(args)
            """
            if engine == 1:
                selected_engine = self.engine1
            elif engine == 2:
                selected_engine = self.engine2
            else:
                raise ValueError("Engine must be 1 or 2")

            with selected_engine.connect() as connection:
                df = pd.read_sql(sql_query, connection)
            return df

    # Database Connection Configuration
    tns_admin_path = r'C:\Oracle2\instantclient_21_13\network\admin'
    credentials_path_db1 = r'\\10.161.85.66\Home\Share\Alteryx_Admin\Configuration\Connection\db_config_main.json'
    credentials_path_db2 = r'\\10.161.85.66\Home\Share\Alteryx_Admin\Configuration\Connection\db_config_datamart.json'
    db_handler = DatabaseHandler(tns_admin_path, credentials_path_db1, credentials_path_db2)

    #Last business day
    with db_handler.engine2.connect() as connection:
        acctcommon = text("""
        SELECT
            a.ACCTNBR,
            a.PRODUCT,
            a.OWNERSORTNAME,
            a.LOANOFFICER,
            a.NOTEBAL,
            a.BOOKBALANCE,
            a.CURRACCTSTATCD,
            a.MJACCTTYPCD,
            a.EFFDATE
        FROM 
            COCCDM.WH_ACCTCOMMON_ME a
        WHERE
            CURRACCTSTATCD IN ('ACT','NPFM')
            AND MJACCTTYPCD IN ('CML','CNS','MTG')


        """)
        start_time = time.time()
        acctcommon = pd.read_sql(acctcommon, connection)
        print("Loaded ACCTCOMMON dataframe in")

        acctloan = text("""
        SELECT
            a.ACCTNBR,
            a.CURRDUEDATE,
            a.COBAL,
            a.TOTALPAYMENTSDUE,
            a.TOTALPIDUE,
            a.TOTALPCTSOLD,
            a.RISKRATINGCD
        FROM 
            COCCDM.WH_ACCTLOAN_ME a
        """)
        start_time = time.time()
        acctloan = pd.read_sql(acctloan, connection)
        print("Loaded ACCTLOAN dataframe in")

        totalpaymentsdue = text("""
        SELECT
            *
        FROM 
            COCCDM.WH_TOTALPAYMENTSDUE a
        """)
        start_time = time.time()
        totalpaymentsdue = pd.read_sql(totalpaymentsdue, connection)
        print("Loaded WH_TOTALPAYMENTSDUE dataframe in")

    with db_handler.engine1.connect() as connection:
        acctstatistichist = text("""
        SELECT 
            *
        FROM 
            OSIBANK.ACCTSTATISTICHIST
        """)
        start_time = time.time()
        acctstatistichist = pd.read_sql(acctstatistichist, connection)
        print(f"ACCTSTATISTICHIST took {time.time() - start_time} seconds.")

        
        data = {
            'acctcommon': acctcommon,
            'acctloan':acctloan,
            'acctstatistichist': acctstatistichist,
            'totalpaymentsdue': totalpaymentsdue
        }
        
        return data





#################################
def acctstatistichist_cleaning(df, acctcommon):
    """ 
    Cleans acctstatistichist table and adds new fields for filtering
    
    Args:
        df: ACCTSTATISTICHIST table (COCC)
        acctcommon: WH_ACCTCOMMON table (COCC)
            - Used for current date
    
    Returns:
        df: ACCSTATISTICHIST with new calculated date fields
            - 'event_date': date (month) of event occurance
            - 'current_date': current_date
            - 'year_start': First day of year (used for YTD calculations)
            - 'year_ago_date': Today's date minus 1 year (for TTM calculations)
        
    Operations:
        - monthcd zero fill 2 digits
        - monthcd to string type
        - yearnbr to string type
        - event_date field = df['yearnbr'] + "-" + df['monthcd'] + "-01"
        - current_date == First record in EFFDATE field from acctcommon table
            -> this is appended to the dataframe as 'current_date' column
        - year_start = current_date year + '01-01'
        - year_ago_date = current_date - 1 year
    """
    df['monthcd'] = df['monthcd'].str.zfill(2)
    df['monthcd'] = df['monthcd'].astype(str)
    df['yearnbr'] = df['yearnbr'].astype(str)
    df['event_date'] = df['yearnbr'] + "-" + df['monthcd'] + "-01"
    df['event_date'] = pd.to_datetime(df['event_date'])
    
    current_date = acctcommon['effdate'][0]
    df['current_date'] = current_date
    
    df['year_start'] = pd.to_datetime(df['current_date'].dt.year.astype(str) + '-01-01')
    df['year_ago_date'] = df['current_date'] - pd.DateOffset(years=1)
    
    return df


#################################



#################################
def count_pd(df, flag):
    """
    This will count past due flags on the account
    
    Args:
        df: ACCTSTATISTICHIST table (COCC)
        flag (str): code to filter on (PD30, PD60, etc...)
        
    Returns:
        df: two column dataframe:
            - acctnbr
            - flag (PD30/60/90/120)
    
    Operations:
        - Filter to only YTD statistics
        - Filter to specified code (ie: PD30 or PD60)
        - Group by acctnbr and sum the number of PD flags for set interval
        - Rename column to match PD Flag

    """
    # Filter to only YTD statistics
    ytd_df = df[df['event_date'] >= df['year_start']]
    
    # Filter to specified code (ie: PD30 or PD60)
    df = ytd_df[ytd_df['statistictypcd'].isin([flag])]
    
    # Group by acctnbr and sum the number of PD flags for set interval
    df = df.groupby('acctnbr')['statisticcount'].sum().reset_index()
    
    # Rename column to match PD Flag
    df = df.rename(columns={'statisticcount': flag})

    
    return df


#################################



#################################
def isolate_total_past_due(df):
    df['totaldue'] = (
        df['principaldue'] +
        df['interestdue'] + 
        df['escrowdue'] +
        df['latechargedue'] +
        df['notedue'] +
        df['miscdue']
    )
    
    # Create delinquent date (due date + 15 days)
    df['delinquentdate'] = df['duedate']
    
    # Exclude payments billed that are not past due & drop helper field
    df = df[df['delinquentdate'] <= df['rundate']]
    df = df.drop(columns=['delinquentdate'])
    
    # Group by acctnbr, sum totaldue and show earliest due date
    aggregated_df = df.groupby('acctnbr').agg({
        'totaldue':'sum',
        'duedate': 'min'
    }).reset_index()
    
    return aggregated_df


#################################




#################################
def merging_tables(acctcommon, acctloan, totalpayments):
    """
    Merging ACCTCOMMON, ACCTLOAN
    """
    # Merging
    merged_df = pd.merge(acctcommon, acctloan, how='left', on='acctnbr')
    merged_df = pd.merge(merged_df, totalpayments, how='left', on='acctnbr')
    
    # Calculate days past due
    current_date = pd.to_datetime('today')
    prior_month_end = pd.to_datetime(current_date.replace(day=1)) - pd.Timedelta(days=1)
    merged_df['effdate'] = prior_month_end
    merged_df['ndpd'] = merged_df['effdate'] - merged_df['currduedate']
    merged_df['ndpd'] = merged_df['ndpd'].dt.days
    merged_df['ndpd'] = np.where(merged_df['ndpd'] < 0, 0, merged_df['ndpd'])
    merged_df['ndpd'] = np.where(merged_df['ndpd'].isnull(), 0, merged_df['ndpd'])

    # Fill null dates with None
    merged_df['currduedate'] = merged_df['currduedate'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)
    
    # Calculate Net Balance
    merged_df['net balance'] = merged_df['bookbalance'] - merged_df['cobal']
    
    # Fill null riskratings
    merged_df['riskratingcd'] = merged_df['riskratingcd'].fillna('Unknown')
    
    # Non accrual field creation
    merged_df['nonaccrual'] = np.where(merged_df['curracctstatcd'] == 'NPFM','Yes','No')
    
    # Exclude Down Payment assistance products
    merged_df = merged_df[~merged_df['product'].str.contains('BCSB Down Payment Assistance',case=False,na=False)]

    return merged_df






#################################
def append_pd_stats(df, pd30, pd60, pd90, pd120):
    """
    Appending pd statistics to main dataframe
    """
    # Merging
    df = pd.merge(df, pd30, how='left', on='acctnbr')
    df = pd.merge(df, pd60, how='left', on='acctnbr')
    df = pd.merge(df, pd90, how='left', on='acctnbr')
    df = pd.merge(df, pd120, how='left', on='acctnbr')
    
    # Fill n/a values
    df['PD30'] = df['PD30'].fillna(0)
    df['PD60'] = df['PD60'].fillna(0)
    df['PD90'] = df['PD90'].fillna(0)
    df['PD12'] = df['PD12'].fillna(0)
    
    # Rename columns
    df = df.rename(columns={
        'PD30': 'YTD Over 30',
        'PD60': 'YTD Over 60',
        'PD90': 'YTD Over 90',
        'PD12': 'YTD Over 120',
    })
    
    return df
    




#################################
def create_validation_model(df):
    """
    Here we explicitly validate all data types and fields that will be used in this report.
    """
    class DelinquencyReport(BaseModel):
        acctnbr: int
        product: str
        ownersortname: str
        loanofficer: Optional[str] = Field(default=None)
        ytd_30: int = Field(alias='YTD Over 30')
        ytd_60: int = Field(alias='YTD Over 60')
        ytd_90: int = Field(alias='YTD Over 90')
        ytd_120: int = Field(alias='YTD Over 120')
        ndpd: int
        bookbalance: float
        cobal: float
        netbalance: float = Field(alias='net balance')
        riskratingcd: str
        nonaccrual: str
        totaldue: float
        nextpaymentdate: Optional[date] = Field(default=None, alias='currduedate')
        mjaccttypcd: str
    
    class DelinquencyDataFrame(BaseModel):
        data: List[DelinquencyReport]
    
    data_dicts = df[[
        'acctnbr',
        'product',
        'ownersortname',
        'loanofficer',
        'YTD Over 30',
        'YTD Over 60',
        'YTD Over 90',
        'YTD Over 120',
        'ndpd',
        'bookbalance',
        'cobal',
        'net balance',
        'riskratingcd',
        'nonaccrual',
        'totaldue',
        'currduedate',
        'mjaccttypcd'
    ]].to_dict('records')
    
    pydantic_model = DelinquencyDataFrame(data=[DelinquencyReport(**row) for row in data_dicts])
    
    return pydantic_model

def unpack_validation_model(x):
    """
    Post-validation, ready to unpack the model and use data to generate report
    """
    temp_dict = x.model_dump()
    df = pd.DataFrame(temp_dict['data'])
    return df




#################################
def section_creation(df, condition, subtitle, title=None):
    """
    Creates the framework for the repeated sections of the delinquency report. This will be called to loop through sections with different filters
    """
    # Data
    df = df.loc[condition].copy().reset_index(drop=True)
    df = df.sort_values(by=['ndpd','ownersortname'], ascending=[False, True])
    df['Tag Type'] = 'data'
    
    # Subtitle
    subtitle_row = pd.DataFrame([{col: '' for col in df.columns}])
    subtitle_row['acctnbr'] = subtitle
    subtitle_row['Tag Type'] = 'subtitle'
    
    # Sum row
    sum_row = pd.DataFrame([{col: '' for col in df.columns}])
    sum_row['acctnbr'] = f'TOTAL {subtitle}'
    sum_row['ndpd'] = len(df)
    sum_row['bookbalance'] = df['bookbalance'].sum()
    sum_row['cobal'] = df['cobal'].sum()
    sum_row['netbalance'] = df['netbalance'].sum()
    sum_row['Tag Type'] = 'sum'
    
    # Title
    if title is not None:
        title_row = pd.DataFrame([{col: '' for col in df.columns}])
        title_row['acctnbr'] = title
        title_row['Tag Type'] = 'title'
        combined_df = pd.concat([title_row, subtitle_row, df, sum_row], ignore_index=True)
    else:
        combined_df = pd.concat([subtitle_row, df, sum_row], ignore_index=True)
    
    return combined_df



# %%
# Main execution


In [2]:
data = retrieve_data()
cleaned_acctstats = acctstatistichist_cleaning(data['acctstatistichist'], data['acctcommon'])
pd30 = count_pd(cleaned_acctstats, 'PD30')
pd60 = count_pd(cleaned_acctstats, 'PD60')
pd90 = count_pd(cleaned_acctstats, 'PD90')
pd120 = count_pd(cleaned_acctstats, 'PD12')
totalpayments = isolate_total_past_due(data['totalpaymentsdue'])
merged_df = merging_tables(data['acctcommon'], data['acctloan'], totalpayments)
df = append_pd_stats(merged_df, pd30, pd60, pd90, pd120)


validation_model = create_validation_model(df)
df = unpack_validation_model(validation_model)


Loaded ACCTCOMMON dataframe in
Loaded ACCTLOAN dataframe in
Loaded WH_TOTALPAYMENTSDUE dataframe in
ACCTSTATISTICHIST took 275.4468574523926 seconds.


In [3]:





# Consumer Loans - 90+
condition = (
    (df['product'].isin([
        'Unsecured Loans',
        'Unsecured HEAT Loans',
        "Flexi PRA's",
        'TMLP Saves Loan '
    ])) & 
    (df['ndpd'] >= 90))
cns90plus = section_creation(df, condition, '90+ DAYS PAST DUE', 'CONSUMER LOANS')


#################################
# Consumer Loans - 60-89
condition = (
    (df['product'].isin([
        'Unsecured Loans',
        'Unsecured HEAT Loans',
        "Flexi PRA's",
        'TMLP Saves Loan '
    ])) & 
    (df['ndpd'] >= 60) & 
    (df['ndpd'] <= 89))
cns60to89 = section_creation(df, condition, '60-89 DAYS PAST DUE')


#################################
# Consumer Loans - 30-59
condition = (
    (df['product'].isin([
        'Unsecured Loans',
        'Unsecured HEAT Loans',
        "Flexi PRA's",
        'TMLP Saves Loan '
    ])) & 
    (df['ndpd'] >= 30) & 
    (df['ndpd'] <= 59))
cns30to59 = section_creation(df, condition, '30-59 DAYS PAST DUE')


#################################
# Consumer Loans - 15-29
condition = (
    (df['product'].isin([
        'Unsecured Loans',
        'Unsecured HEAT Loans',
        "Flexi PRA's",
        'TMLP Saves Loan '
    ])) & 
    (df['ndpd'] >= 15) & 
    (df['ndpd'] <= 29))
cns15to29 = section_creation(df, condition, '15-29 DAYS PAST DUE')


#################################
# HELOC - 90+
condition = (
    (df['product'].isin([
        'Equity Line of Credit',
        'Special HELOC'
    ])) & 
    (df['ndpd'] >= 90))
heloc90plus = section_creation(df, condition, '90+ DAYS PAST DUE', 'HOME EQUITY LINE OF CREDIT')


#################################
# HELOC - 60-89
condition = (
    (df['product'].isin([
        'Equity Line of Credit',
        'Special HELOC'
    ])) & 
    (df['ndpd'] >= 60) &
    (df['ndpd'] <= 89))
heloc60to89 = section_creation(df, condition, '60-89 DAYS PAST DUE')


#################################
# HELOC - 30-59
condition = (
    (df['product'].isin([
        'Equity Line of Credit',
        'Special HELOC'
    ])) & 
    (df['ndpd'] >= 30) & 
    (df['ndpd'] <= 59))
heloc30to59 = section_creation(df, condition, '30-59 DAYS PAST DUE')


#################################
# HELOC - 15-29
condition = (
    (df['product'].isin([
        'Equity Line of Credit',
        'Special HELOC'
    ])) & 
    (df['ndpd'] >= 15) & 
    (df['ndpd'] <= 29))
heloc15to29 = section_creation(df, condition, '15-29 DAYS PAST DUE')


#################################
# Indirect - 90+
condition = (
    ((df['product'].str.contains('Indirect',case=False)) | (df['product'] == 'BCSB Auto Used')) & 
    (df['ndpd'] >= 90))
indirect90plus = section_creation(df, condition,'90+ DAYS PAST DUE', 'INDIRECT LOANS')


#################################
# Indirect - 60-89
condition = (
    ((df['product'].str.contains('Indirect',case=False)) | (df['product'] == 'BCSB Auto Used')) & 
    (df['ndpd'] >= 60) &
    (df['ndpd'] <= 89))
indirect60to89 = section_creation(df, condition,'60-89 DAYS PAST DUE')


#################################
# Indirect - 30-59
condition = (
    ((df['product'].str.contains('Indirect',case=False)) | (df['product'] == 'BCSB Auto Used')) & 
    (df['ndpd'] >= 30) &
    (df['ndpd'] <= 59))
indirect30to59 = section_creation(df, condition,'30-59 DAYS PAST DUE')


#################################
# Indirect - 15-29
condition = (
    ((df['product'].str.contains('Indirect',case=False)) | (df['product'] == 'BCSB Auto Used')) & 
    (df['ndpd'] >= 15) &
    (df['ndpd'] <= 29))
indirect15to29 = section_creation(df, condition,'15-29 DAYS PAST DUE')


#################################
# Residential - 90+
condition = (
    (df['mjaccttypcd'] == 'MTG') &
    (~df['product'].isin(['Equity Line of Credit','Special HELOC'])) & 
    (df['ndpd'] >= 90))
resi90plus = section_creation(df, condition,'90+ DAYS PAST DUE', 'RESIDENTIAL MORTGAGES')


#################################
# Residential - 60-89
condition = (
    (df['mjaccttypcd'] == 'MTG') &
    (~df['product'].isin(['Equity Line of Credit','Special HELOC'])) & 
    (df['ndpd'] >= 60) &
    (df['ndpd'] <= 89))
resi60to89 = section_creation(df, condition,'60-89 DAYS PAST DUE')


#################################
# Residential - 30-59
condition = (
    (df['mjaccttypcd'] == 'MTG') &
    (~df['product'].isin(['Equity Line of Credit','Special HELOC'])) & 
    (df['ndpd'] >= 30) &
    (df['ndpd'] <= 59))
resi30to59 = section_creation(df, condition,'30-59 DAYS PAST DUE')


#################################
# Residential - 15-29
condition = (
    (df['mjaccttypcd'] == 'MTG') &
    (~df['product'].isin(['Equity Line of Credit','Special HELOC'])) & 
    (df['ndpd'] >= 15) &
    (df['ndpd'] <= 29))
resi15to29 = section_creation(df, condition,'15-29 DAYS PAST DUE')


#################################
# Commercial - 90+
condition = (
    (df['mjaccttypcd'] == 'CML') &
    (~df['product'].str.contains('Indirect',case=False,na=False)) &
    (df['netbalance'] != 0) &
    (df['ndpd'] >= 90))
cml90plus = section_creation(df, condition,'90+ DAYS PAST DUE', 'COMMERCIAL LOANS')


#################################
# Commercial - 60-89
condition = (
    (df['mjaccttypcd'] == 'CML') &
    (~df['product'].str.contains('Indirect',case=False,na=False)) &
    (df['netbalance'] != 0) &
    (df['ndpd'] >= 60) &
    (df['ndpd'] <= 89))
cml60to89 = section_creation(df, condition,'60-89 DAYS PAST DUE')


#################################
# Commercial - 30-59
condition = (
    (df['mjaccttypcd'] == 'CML') &
    (~df['product'].str.contains('Indirect',case=False,na=False)) &
    (df['netbalance'] != 0) &
    (df['ndpd'] >= 30) &
    (df['ndpd'] <= 59))
cml30to59 = section_creation(df, condition,'30-59 DAYS PAST DUE')


#################################
# Commercial - 15-29
condition = (
    (df['mjaccttypcd'] == 'CML') &
    (~df['product'].str.contains('Indirect',case=False,na=False)) &
    (df['netbalance'] != 0) &
    (df['ndpd'] >= 15) &
    (df['ndpd'] <= 29))
cml15to29 = section_creation(df, condition,'15-29 DAYS PAST DUE')


#################################
# PEM - 90+
condition = (
    (df['riskratingcd'] == '3P') &
    (df['ndpd'] >= 90))
pem90plus = section_creation(df, condition,'90+ DAYS PAST DUE', 'COMMERCIAL LOANS - PORTFOLIO EXCEPTIONS MANAGED')


#################################
# PEM - 60-89
condition = (
    (df['riskratingcd'] == '3P') &
    (df['ndpd'] >= 60) &
    (df['ndpd'] <= 89))
pem60to89 = section_creation(df, condition,'60-89 DAYS PAST DUE')


#################################
# PEM - 30-59
condition = (
    (df['riskratingcd'] == '3P') &
    (df['ndpd'] >= 30) &
    (df['ndpd'] <= 59))
pem30to59 = section_creation(df, condition,'30-59 DAYS PAST DUE')


#################################
# PEM - 15-29
condition = (
    (df['riskratingcd'] == '3P') &
    (df['ndpd'] >= 15) &
    (df['ndpd'] <= 29))
pem15to29 = section_creation(df, condition,'15-29 DAYS PAST DUE')


#################################
# Commercial Reduced to $0
condition = (
    (df['mjaccttypcd'] == 'CML') &
    (df['netbalance'] == 0))
reducedtozero = section_creation(df, condition, '', 'NET BALANCE PAID or REDUCED TO $0')


#################################
# Repossessed
condition = (
    (df['product'].str.contains('Repossessed Collateral', case=False))
)
repo = section_creation(df, condition, '', 'REPOSSESSED')



In [5]:

#################################
# Here all dfs are stored in a list
all_dfs = [
    cns90plus,
    cns60to89,
    cns30to59,
    cns15to29,
    heloc90plus,
    heloc60to89,
    heloc30to59,
    heloc15to29,
    indirect90plus,
    indirect60to89,
    indirect30to59,
    indirect15to29,
    resi90plus,
    resi60to89,
    resi30to59,
    resi15to29,
    cml90plus,
    cml60to89,
    cml30to59,
    cml15to29,
    pem90plus,
    pem60to89,
    pem30to59,
    pem15to29,
    reducedtozero,
    repo]

concat_dfs = [] # Empty list to store dataframes

for i,df in enumerate(all_dfs):
    concat_dfs.append(df) # Each dataframe
    if i < len(all_dfs) - 1:
        concat_dfs.append(pd.DataFrame([[None]*df.shape[1]], columns=df.columns)) # Null row
    
completed_df = pd.concat(concat_dfs, ignore_index=True)


#################################
# Rename Columns
completed_df = completed_df.rename(columns={
    'acctnbr': 'Account Number',
    'product': 'Product Name',
    'ownersortname': 'Customer Name',
    'loanofficer': 'Responsibility Officer',
    'ytd_30': 'YTD Over 30',
    'ytd_60': 'YTD Over 60',
    'ytd_90': 'YTD Over 90',
    'ytd_120': 'YTD Over 120',
    'ndpd': 'NDPD',
    'bookbalance': 'Current Balance',
    'cobal': 'Charged Off',
    'netbalance': 'Net Balance',
    'riskratingcd': 'Risk',
    'nonaccrual': 'Non Accrual',
    'totaldue': 'Total Past Due',
    'nextpaymentdate': 'Next Payment Date'
})


#################################
# Write to Excel

# Most recent month end
today = datetime.today()
if today.day == 1:
    most_recent_month_end = today - timedelta(days=1)
else:
    most_recent_month_end = today.replace(day=1) - timedelta(days=1)
most_recent_month_end_str = most_recent_month_end.strftime("%m/%d/%y")
me_date_no_slash = most_recent_month_end.strftime("%m%d%y") # Month End date to append to file name

## Set Directory and Filename
output_dir = r"\\10.161.85.66\Home\Share\Data & Analytics Initiatives\Project Management\Chad Projects\Resolution Committee Automation\Delinquency\Production\Output"
os.makedirs(output_dir, exist_ok=True)
filename = f"Delinquency_{me_date_no_slash}.xlsx"
output_file = os.path.join(output_dir,filename)

with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
    completed_df.to_excel(writer, index=False, sheet_name='Sheet1', startrow=4)

wb = load_workbook(output_file)
ws = wb.active

## Define Formatting
# Dimensions
ws.row_dimensions[5].height = 44

ws.column_dimensions['A'].width = 13
ws.column_dimensions['B'].width = 29
ws.column_dimensions['C'].width = 32
ws.column_dimensions['D'].width = 24
ws.column_dimensions['E'].width = 5
ws.column_dimensions['F'].width = 5
ws.column_dimensions['G'].width = 5
ws.column_dimensions['H'].width = 5
ws.column_dimensions['I'].width = 6
ws.column_dimensions['J'].width = 12
ws.column_dimensions['K'].width = 10
ws.column_dimensions['L'].width = 12
ws.column_dimensions['M'].width = 29
ws.column_dimensions['N'].width = 8
ws.column_dimensions['O'].width = 13
ws.column_dimensions['P'].width = 10
ws.column_dimensions['Q'].width = 46



# Font
upper_section_font = Font(size=14, name='Calibri', bold=True, italic=True)
title_font = Font(size=12, name='Arial', bold=True)
subtitle_font = Font(size=10, name='Arial', bold=True)
data_font = Font(size=10, name='Arial')
sum_font = Font(size=10, name='Arial', bold=True)
wrap_alignment = Alignment(wrap_text=True, vertical='bottom', horizontal='center')
comma_format = '#,##0.00'
short_date_format = 'MM/DD/YYYY'


# Border
double_border = Border(bottom=Side(style='double'))

# Header
ws['A1'] = "BRISTOL COUNTY SAVINGS BANK"
ws['A1'].font = upper_section_font
ws['A2'] = "MONTHLY DELINQUENCY REPORT"
ws['A2'].font = upper_section_font
ws['A3'] = f"AS OF {most_recent_month_end_str}"
ws['A3'].font = upper_section_font



# Clean out Column Q
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=17, max_col=17):
    for cell in row:
        cell.value = None
        

# Center Align Columns E:I
for col in range(5,10):
    for cell in ws.iter_cols(min_col=col, max_col=col, min_row=1, max_row=ws.max_row):
        for c in cell:
            c.alignment = Alignment(horizontal='center')
            
# Center Align Columns M:N
for col in range(13,15):
    for cell in ws.iter_cols(min_col=col, max_col=col, min_row=1, max_row=ws.max_row):
        for c in cell:
            c.alignment = Alignment(horizontal='center')

# for cell in ws['P']:
#     cell.number_format = NumberFormat('%m/%d/%y')

## Iterate over rows and apply formatting
for cell in ws[5]:
    cell.alignment = wrap_alignment
    
start_row = 6 # This is a counter variable

for idx, row in completed_df.iterrows():
    row_type = row['Tag Type']
    current_row = start_row + idx
    
    if row_type == 'title':
        for cell in ws[current_row]:
            cell.font = title_font
            
    elif row_type == 'subtitle':
        for cell in ws[current_row]:
            cell.font = subtitle_font
            
    elif row_type == 'data':
        for cell in ws[current_row]:
            cell.font = data_font
            
    elif row_type == 'sum':
        for cell in ws[current_row]:
            cell.font = sum_font
            cell.border = double_border
            
# Adding commas to columns J:L
for col in range(10, 13):
    for row in range(1, ws.max_row + 1):
        cell = ws.cell(row=row, column=col)
        cell.number_format = comma_format
        
# Adding commas to column O
for col in range(15, 16):
    for row in range(1, ws.max_row + 1):
        cell = ws.cell(row=row, column=col)
        cell.number_format = comma_format
        
# Short date format for column P
for col in range(16, 17):
    for row in range(1, ws.max_row + 1):
        cell = ws.cell(row=row, column=col)
        cell.number_format = short_date_format

# Final formatting
# Clean out Column R
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=18, max_col=18):
    for cell in row:
        cell.value = None
        cell.border = None
        
ws['Q5'] = "Comments"
temp_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
ws['Q5'].border = temp_border
            
# Saving
wb.save(output_file)

print(f"Report saved to {output_file}")

# Email
recipients = [
    # "loan.review@bcsbmail.com",
    # "chad.doorley@bcsbmail.com",
    "brandon.george@bcsbmail.com"
]
bcc_recipients = [
    "chad.doorley@bcsbmail.com"
]
outlook = Dispatch("Outlook.Application")
message = outlook.CreateItem(0)
# message.Display()
message.To = ";".join(recipients)
message.BCC = ";".join(bcc_recipients)
message.Subject = f"Delinquency Report - ME: {most_recent_month_end_str}"
message.Body = "Hi, \n\nIgnore the previous version, this has corrected an issue. Attached is the monthly Delinquency report for the Resolution Committee Package. Please let me know if you have any questions."
message.Attachments.Add(str(output_file))
message.Send()
print("Email sent!")




Report saved to \\10.161.85.66\Home\Share\Data & Analytics Initiatives\Project Management\Chad Projects\Resolution Committee Automation\Delinquency\Production\Output\Delinquency_113024.xlsx
Email sent!
